In [1]:
import pandas as pd

import tensorflow as tf
import json
import numpy as np
import pandas as pd
import cv2
from skimage import exposure

import matplotlib.pyplot as plt

from cell_division.nets.transfer_learning import CNN
from auxiliary.data.dataset_cell import CellDataset
from auxiliary import values as v
from auxiliary.utils.colors import bcolors as c
from auxiliary.utils import visualizer as vis
from auxiliary.data import imaging

from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import GlobalAveragePooling2D
from cell_division.nets.custom_layers import (
    w_cel_loss, 
    focal_loss,
    ExtendedLSEPooling,
    extended_w_cel_loss,
    LSEPooling
)


from cell_division.nets.cam import GradCAM, overlay_heatmap, CAM, GradCAMpp

# GPU config
from auxiliary.utils.timer import LoadingBar
from auxiliary.data.dataset_unlabeled import UnlabeledDataset
from auxiliary.data.dataset_ht import HtDataset

In [9]:
# specimens = [item for sublist in v.specimens.values() for item in sublist]
specimens = ['0518_E3']

ht_ds = HtDataset() 

for specimen in specimens:    
    print(f'{c.OKBLUE}Processing specimen:{c.ENDC} {specimen}')

    try:
        # Check if the specimen has the required files
        p_img_raw, _ = ht_ds.read_specimen(specimen, level='Nuclei', type='RawImages')
        p_img_seg, _ = ht_ds.read_specimen(specimen, level='Nuclei', type='Segmentation')
        features = ht_ds.get_features(specimen, type='Nuclei')

        # Read the images
        img_raw = imaging.read_image(p_img_raw, verbose=1).astype(np.uint8)
        img_seg = imaging.read_image(p_img_seg, verbose=1)

        # Get the features
        labels, centroids = features['original_labels'], features['centroids']

        for i, label in enumerate(labels):
            # Get the centroids of the cell
            cen = centroids[i].replace('[', '').replace(']', '').split(',')
            x, y, z = int(cen[0]), int(cen[1]), int(cen[2])

            # Crop the cell region + Remove empty z-slices
            mask = img_seg[x-25: x+25, y-25: y+25, z-10: z+10]
            mask = np.where(mask == label, 255, 0)
            non_empty_slices = [i for i in range(mask.shape[2]) if np.any(mask[..., i])]
            mask = mask[..., non_empty_slices]

            # Crop the cell region of the raw image
            img_cell = img_raw[x-25: x+25, y-25: y+25, z-25: z+25]
            img_cell = img_cell[..., non_empty_slices]

            # Erase the background
            img_cell = np.where(mask == 0, 0, img_cell)

            # Save the images
            img_cell = img_cell.astype(np.uint8)
            if img_cell.shape[2] > 0:
                imaging.save_prediction(
                    img_cell,
                    v.data_path + 'CellDivision/images_unlabeled/' + f'{specimen}_{int(label)}.tif',
                    verbose=1
                )

    except Exception as e:
        print(f'{c.FAIL}Error:{c.ENDC} {e}')
        continue


Processing specimen: 0518_E3
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr5/RawImages/Nuclei/20190518_E3_DAPI_decon_0.5.nii.gz
Reading TIFF: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr5/Segmentation/Nuclei/20190518_E3_Fernando.tif

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/CellDivision/images_unlabeled/0518_E3_27173.tif

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/CellDivision/images_unlabeled/0518_E3_43049.tif

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/CellDivision/images_unlabeled/0518_E3_52265.tif

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/CellDivision/images_unlabeled/0518_E3_47473.tif

Saving prediction: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_M

KeyboardInterrupt: 